In [113]:
import pandas as pd
import numpy as np
from geojson import Polygon, Feature, FeatureCollection
import csv

In [116]:
data = pd.read_csv("community_cases.csv", index_col = 0)
data

,7 Oct to 10 Oct (5),11 Oct to 17 Oct (7),18 Oct to 24 Oct (7),25 Oct to 31 Oct (7),1 Nov to 3 Nov (3)
0-11 years old (1),719,1164,1344,1710,795
12-19 years old (1),401,591,743,841,344
20-39 years old (2),3450,6040,7309,8376,3193
40-59 years old (2),3646,6279,7140,7861,2952
60-69 years old (1),1402,2101,2146,2164,835
70-100 years old (3),1035,1572,1514,1570,645


In [117]:
data = data.rename({"7 Oct to 10 Oct (5)" : "7oct_10oct(5) 50",
            "11 Oct to 17 Oct (7)" : "11oct_17oct(7) 70",
            "18 Oct to 24 Oct (7)" : "18oct_24oct(7) 70",
            "25 Oct to 31 Oct (7)" : "25oct_31oct(7) 70",
            "1 Nov to 3 Nov (3)" : "1nov_3nov(3) 30"
            }, axis = "columns")

In [118]:
data = data.rename({"0-11 years old (1)" : "0_11_old(1) 20",
             "12-19 years old (1)" : "12_19_old(1) 20",
             "20-39 years old (2)" : "20_39_old(2) 40",
             "40-59 years old (2)" : "40_59_old(2) 40",
             "60-69 years old (1)" : "60_69_old(1) 20",
             "70-100 years old (3)" : "70_100_old(3) 60"
            }, axis = "index")
data

,7oct_10oct(5) 50,11oct_17oct(7) 70,18oct_24oct(7) 70,25oct_31oct(7) 70,1nov_3nov(3) 30
0_11_old(1) 20,719,1164,1344,1710,795
12_19_old(1) 20,401,591,743,841,344
20_39_old(2) 40,3450,6040,7309,8376,3193
40_59_old(2) 40,3646,6279,7140,7861,2952
60_69_old(1) 20,1402,2101,2146,2164,835
70_100_old(3) 60,1035,1572,1514,1570,645


In [119]:
def get_points_along_line(start, end, count):
    x_points = np.linspace(start[0], end[0], count)
    y_points = np.linspace(start[1], end[1], count)
    points = []
    for idx, (x, y) in enumerate(zip(x_points, y_points)):
        points.append((x,y))
    points.pop();
    return points

In [120]:
def get_points(bottomleft, width, height, count = 10): # top-left start, clockwise
    points = []
    
    bottomright = [bottomleft[0] + width, bottomleft[1]]
    topright = [bottomright[0], bottomright[1] + height]
    topleft = [topright[0] - width, topright[1]]
    
    points.extend(get_points_along_line(bottomleft, bottomright, count))
    points.extend(get_points_along_line(bottomright, topright, count))    
    points.extend(get_points_along_line(topright, topleft, count))    
    points.extend(get_points_along_line(topleft, bottomleft, count))
    points.append(bottomleft)
    
    return points

In [124]:
widths = []
heights = []
for cidx, col in enumerate(data.columns):
    widths.append(int(col.split()[1]))
    
for ridx, row in enumerate(data.index):
    heights.append(int(row.split()[1]))


In [128]:
widths = (np.array(widths)/sum(widths))*300
widths

array([51.72413793, 72.4137931 , 72.4137931 , 72.4137931 , 31.03448276])

In [129]:
heights = (np.array(heights)/sum(heights))*300
heights

array([30., 30., 60., 60., 30., 90.])

In [130]:
n_points = 30
start = [0,0]
features = []
for cidx, col in enumerate(data.columns):
    width = widths[cidx]
    start[1] = 0 #make row position 0    
    for ridx, row in enumerate(data.index):
        height = heights[ridx]
        name = col.split()[0] + "_" + row.split()[0]
        polygon = Polygon([get_points(start, width, height, n_points)])
        feature = Feature(geometry = polygon, properties = {"name" : name})
        features.append(feature)
        start[1] += height
    start[0] += width #increase col position

In [131]:
gj = FeatureCollection(features)
%store geojson.dumps(gj) >cases.geojson

Writing 'geojson.dumps(gj)' (str) to file 'cases.geojson'.


In [132]:
with open('cases.csv', 'w', encoding='UTF8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name','cases'])
    for cidx, col in enumerate(data.columns):
        for ridx, row in enumerate(data.index):
            name = col.split()[0] + "_" + row.split()[0]
            rw = [name, data.iloc[ridx, cidx]]
            writer.writerow(rw)